In [192]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
#from sklearn.ensemble import GradientBoostingRegressor
#from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score
#from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
#from sklearn.model_selection import cross_val_score, cross_val_predict
#from category_encoders import WOEEncoder, SumEncoder

In [193]:
os.getcwd()

'D:\\ML\\Gewissta\\My course\\Categorical Feature Encoding Challenge II_Kaggle'

In [194]:
os.chdir('D:\\ML\\Gewissta\\My course\\Categorical Feature Encoding Challenge II_Kaggle')

In [195]:
# загрузка
data_train = pd.read_csv('train.csv', sep = ',')
data_test = pd.read_csv('test.csv', sep = ',')

In [196]:
# простой обзор

In [197]:
data_train.head(10)

,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,...,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
0,0,0.0,0.0,0.0,F,N,Red,Trapezoid,Hamster,Russia,...,02e7c8990,3.0,Contributor,Hot,c,U,Pw,6.0,3.0,0
1,1,1.0,1.0,0.0,F,Y,Red,Star,Axolotl,NaN,...,f37df64af,3.0,Grandmaster,Warm,e,X,pE,7.0,7.0,0
2,2,0.0,1.0,0.0,F,N,Red,NaN,Hamster,Canada,...,NaN,3.0,NaN,Freezing,n,P,eN,5.0,9.0,0
3,3,NaN,0.0,0.0,F,N,Red,Circle,Hamster,Finland,...,f9d456e57,1.0,Novice,Lava Hot,a,C,NaN,3.0,3.0,0
4,4,0.0,NaN,0.0,T,N,Red,Triangle,Hamster,Costa Rica,...,c5361037c,3.0,Grandmaster,Cold,h,C,OZ,5.0,12.0,0
5,5,0.0,NaN,1.0,T,N,Red,Triangle,Lion,China,...,05afc0f8b,2.0,Expert,Hot,b,Q,wa,3.0,4.0,0
6,6,0.0,0.0,0.0,F,N,Red,Triangle,Hamster,Costa Rica,...,4d3766412,1.0,Grandmaster,Cold,c,R,rg,5.0,6.0,0
7,7,0.0,0.0,1.0,T,N,Red,Triangle,Axolotl,Finland,...,a651dec43,3.0,Expert,Cold,b,Y,PS,1.0,1.0,0
8,8,0.0,0.0,0.0,F,N,Blue,Polygon,Hamster,Russia,...,NaN,1.0,Novice,Boiling Hot,c,N,mX,6.0,3.0,0
9,9,0.0,0.0,NaN,F,Y,Red,Polygon,Hamster,Finland,...,79b29d54c,3.0,Contributor,Lava Hot,n,I,OZ,1.0,8.0,1


In [198]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600000 entries, 0 to 599999
Data columns (total 25 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   id      600000 non-null  int64  
 1   bin_0   582106 non-null  float64
 2   bin_1   581997 non-null  float64
 3   bin_2   582070 non-null  float64
 4   bin_3   581986 non-null  object 
 5   bin_4   581953 non-null  object 
 6   nom_0   581748 non-null  object 
 7   nom_1   581844 non-null  object 
 8   nom_2   581965 non-null  object 
 9   nom_3   581879 non-null  object 
 10  nom_4   581965 non-null  object 
 11  nom_5   582222 non-null  object 
 12  nom_6   581869 non-null  object 
 13  nom_7   581997 non-null  object 
 14  nom_8   582245 non-null  object 
 15  nom_9   581927 non-null  object 
 16  ord_0   581712 non-null  float64
 17  ord_1   581959 non-null  object 
 18  ord_2   581925 non-null  object 
 19  ord_3   582084 non-null  object 
 20  ord_4   582070 non-null  object 
 21  ord_5   58

In [199]:
data_test.head(10)

,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,...,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month
0,600000,0.0,0.0,0.0,F,Y,Blue,Polygon,Axolotl,Finland,...,ca9ad1d4b,fced9e114,3.0,Novice,Boiling Hot,f,U,oU,3.0,9.0
1,600001,0.0,0.0,0.0,F,Y,Red,Circle,Lion,Russia,...,060a21580,7ca8775da,1.0,Novice,Cold,n,N,NaN,2.0,8.0
2,600002,0.0,0.0,0.0,F,Y,Blue,Circle,Axolotl,Russia,...,165e81a00,5940334c9,1.0,Expert,Warm,i,N,DN,2.0,6.0
3,600003,1.0,0.0,0.0,F,N,Red,Polygon,Axolotl,Costa Rica,...,77d41330d,6fbdeefc8,1.0,Expert,Hot,m,B,AG,1.0,6.0
4,600004,0.0,0.0,1.0,F,Y,Red,Circle,NaN,Finland,...,2218d9dfe,2a27c8fde,1.0,Contributor,Lava Hot,o,J,DT,3.0,3.0
5,600005,0.0,0.0,0.0,NaN,Y,Red,Star,Axolotl,Costa Rica,...,745b909d1,30db15878,3.0,Grandmaster,Cold,e,Y,th,5.0,6.0
6,600006,0.0,0.0,0.0,F,Y,Red,Trapezoid,Axolotl,Russia,...,75a957c6d,ecbea3fb9,2.0,Novice,Lava Hot,n,U,RD,1.0,6.0
7,600007,0.0,0.0,1.0,F,N,Red,Trapezoid,Axolotl,India,...,5bdcf7324,48ca7cb85,1.0,Grandmaster,Boiling Hot,f,NaN,pT,2.0,6.0
8,600008,0.0,0.0,0.0,T,Y,Red,Triangle,Dog,Russia,...,8aeac2495,e663c4eec,2.0,Master,Warm,c,Y,nj,5.0,5.0
9,600009,0.0,0.0,0.0,F,N,Green,Polygon,Lion,Costa Rica,...,220190c9e,0ce4b3b6e,2.0,Novice,Hot,k,M,dp,7.0,2.0


In [200]:
data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400000 entries, 0 to 399999
Data columns (total 24 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   id      400000 non-null  int64  
 1   bin_0   388099 non-null  float64
 2   bin_1   387962 non-null  float64
 3   bin_2   388028 non-null  float64
 4   bin_3   388049 non-null  object 
 5   bin_4   388049 non-null  object 
 6   nom_0   387938 non-null  object 
 7   nom_1   388053 non-null  object 
 8   nom_2   387821 non-null  object 
 9   nom_3   387824 non-null  object 
 10  nom_4   388007 non-null  object 
 11  nom_5   388088 non-null  object 
 12  nom_6   387988 non-null  object 
 13  nom_7   387997 non-null  object 
 14  nom_8   388044 non-null  object 
 15  nom_9   387940 non-null  object 
 16  ord_0   388107 non-null  float64
 17  ord_1   387833 non-null  object 
 18  ord_2   387895 non-null  object 
 19  ord_3   387947 non-null  object 
 20  ord_4   388067 non-null  object 
 21  ord_5   38

In [201]:
#количество категорий в train
data_train.nunique()

id        600000
bin_0          2
bin_1          2
bin_2          2
bin_3          2
bin_4          2
nom_0          3
nom_1          6
nom_2          6
nom_3          6
nom_4          4
nom_5       1220
nom_6       1519
nom_7        222
nom_8        222
nom_9       2218
ord_0          3
ord_1          5
ord_2          6
ord_3         15
ord_4         26
ord_5        190
day            7
month         12
target         2
dtype: int64

In [202]:
#количество категорий в test
data_test.nunique()

id       400000
bin_0         2
bin_1         2
bin_2         2
bin_3         2
bin_4         2
nom_0         3
nom_1         6
nom_2         6
nom_3         6
nom_4         4
nom_5      1219
nom_6      1517
nom_7       222
nom_8       222
nom_9      2216
ord_0         3
ord_1         5
ord_2         6
ord_3        15
ord_4        26
ord_5       190
day           7
month        12
dtype: int64

In [203]:
# имена всех столбцов без 'id', 'target'
predict_columns = data_train.columns.tolist()
predict_columns.remove('id')
predict_columns.remove('target')

In [204]:
# смотрим редкие категории для признаков с небольшим количеством категорий (low-cardinality)
predict_columns_noncontinuous = predict_columns
predict_columns_unneccessary = ['nom_5', 'nom_6', 'nom_7', 'nom_8', 'nom_9']
for i in predict_columns_unneccessary:
    predict_columns_noncontinuous.remove(i)
for i in predict_columns_noncontinuous:
    print(data_train[i].value_counts(dropna=False))

0.0    528377
1.0     53729
NaN     17894
Name: bin_0, dtype: int64
0.0    474018
1.0    107979
NaN     18003
Name: bin_1, dtype: int64
0.0    419845
1.0    162225
NaN     17930
Name: bin_2, dtype: int64
F      366212
T      215774
NaN     18014
Name: bin_3, dtype: int64
N      312344
Y      269609
NaN     18047
Name: bin_4, dtype: int64
Red      323286
Blue     205861
Green     52601
NaN       18252
Name: nom_0, dtype: int64
Triangle     164190
Polygon      152563
Trapezoid    119438
Circle       104995
Square        26503
NaN           18156
Star          14155
Name: nom_1, dtype: int64
Hamster    164897
Axolotl    152319
Lion       119504
Dog        104825
Cat         26276
NaN         18035
Snake       14144
Name: nom_2, dtype: int64
India         164869
Costa Rica    151827
Russia        119840
Finland       104601
Canada         26425
NaN            18121
China          14317
Name: nom_3, dtype: int64
Theremin    308621
Bassoon     196639
Oboe         49996
Piano        26709
NaN

In [205]:
# редких категорий нет

In [206]:
# смотрим пропуски в train
data_train.isnull().sum()

id            0
bin_0     17894
bin_1     18003
bin_2     17930
bin_3     18014
bin_4     18047
nom_0     18252
nom_1     18156
nom_2     18035
nom_3     18121
nom_4     18035
nom_5     17778
nom_6     18131
nom_7     18003
nom_8     17755
nom_9     18073
ord_0     18288
ord_1     18041
ord_2     18075
ord_3     17916
ord_4     17930
ord_5     17713
day       17952
month     17988
target        0
dtype: int64

In [207]:
# смотрим пропуски в train
data_test.isnull().sum()

id           0
bin_0    11901
bin_1    12038
bin_2    11972
bin_3    11951
bin_4    11951
nom_0    12062
nom_1    11947
nom_2    12179
nom_3    12176
nom_4    11993
nom_5    11912
nom_6    12012
nom_7    12003
nom_8    11956
nom_9    12060
ord_0    11893
ord_1    12167
ord_2    12105
ord_3    12053
ord_4    11933
ord_5    12047
day      12025
month    11984
dtype: int64

In [208]:
# Копируем файлы для дальнейшей работы
train = data_train.copy()
test = data_test.copy()

In [209]:
# Данных много. Просто удалим все строки с пропусками. Картина не должна сильно пострадать.
train_dropna = train.dropna().copy()
test_dropna = test.dropna().copy()

In [210]:
#количество категорий в train после удаления пропусков
train_dropna.nunique()

id        298042
bin_0          2
bin_1          2
bin_2          2
bin_3          2
bin_4          2
nom_0          3
nom_1          6
nom_2          6
nom_3          6
nom_4          4
nom_5       1219
nom_6       1519
nom_7        222
nom_8        222
nom_9       2217
ord_0          3
ord_1          5
ord_2          6
ord_3         15
ord_4         26
ord_5        190
day            7
month         12
target         2
dtype: int64

In [211]:
#количество категорий в test после удаления пропусков
test_dropna.nunique()

id       198162
bin_0         2
bin_1         2
bin_2         2
bin_3         2
bin_4         2
nom_0         3
nom_1         6
nom_2         6
nom_3         6
nom_4         4
nom_5      1219
nom_6      1514
nom_7       222
nom_8       222
nom_9      2208
ord_0         3
ord_1         5
ord_2         6
ord_3        15
ord_4        26
ord_5       190
day           7
month        12
dtype: int64

In [212]:
# Если картина и пострадала в контексте категорий, то не сильно

In [213]:
# очевидные преобразования бинарных признаков

map_binTF = {"T":1,"F":0}
map_binYN = {"Y":1,"N":0}
train_dropna['bin_3'] = train_dropna['bin_3'].map(map_binTF)
train_dropna['bin_4'] = train_dropna['bin_4'].map(map_binYN)
test_dropna['bin_3'] = test_dropna['bin_3'].map(map_binTF)
test_dropna['bin_4'] = test_dropna['bin_4'].map(map_binYN)

In [214]:
# очевидные преобразования номинальных признаков с небольшим количеством категорий (low-cardinality)
train_dropna = pd.get_dummies(train_dropna, columns=['nom_0', 'nom_1', 'nom_2', 'nom_3', 'nom_4'])
test_dropna = pd.get_dummies(test_dropna, columns=['nom_0', 'nom_1', 'nom_2', 'nom_3', 'nom_4'])

In [215]:
# преобразования номинальных признаков с большим количеством категорий (high-cardinality)
import h2o
h2o.init(nthreads=-1, max_mem_size=8)

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,3 hours 27 mins
H2O_cluster_timezone:,Asia/Yekaterinburg
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.2
H2O_cluster_version_age:,3 months and 11 days !!!
H2O_cluster_name:,H2O_from_python_Dell_5en0br
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.749 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


In [216]:
train_h2o = h2o.H2OFrame(train_dropna)
test_h2o = h2o.H2OFrame(test_dropna)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [217]:
#train_h2o.describe()

In [218]:
train_h2o['target'] = train_h2o['target'].asfactor()
#test_h2o['target'] = test_h2o['target'].asfactor()

In [219]:
#train_h2o.describe()

In [220]:
from h2o.targetencoder import TargetEncoder

# создаем столбец с индексами блоков
train_h2o['cv_fold_te'] = train_h2o.kfold_column(n_folds=5, seed=42)

# создаем экземпляр класса TargetEncoder (модель)
# для обучающей выборки
targetEncoder = TargetEncoder(x=['nom_5', 'nom_6', 'nom_7', 'nom_8', 'nom_9'], 
                              blended_avg=True, 
                              inflection_point=3, 
                              smoothing=1,
                              y='target', 
                              fold_column='cv_fold_te')

# обучаем модель, т.е. создаем таблицу, в соответствии с которой 
# категориям предиктора в обучающей выборке будут сопоставлены 
# сглаженные средние значения зависимой переменной
targetEncoder.fit(train_h2o)

In [221]:
# применяем модель к обучающей выборке, категории предиктора 
# в обучающей выборке заменяются на сглаженные средние 
# значения зависимой переменной
train_h2o = targetEncoder.transform(frame=train_h2o, 
                                   holdout_type='kfold',
                                   noise=0
                                  )

In [222]:
#train_h2o.head()

In [223]:
train_h2o = train_h2o.as_data_frame()
test_h2o = test_h2o.as_data_frame()

In [224]:
# создание словарей для map() (или replace()) в test_h2o

train_h2o_nom_5_dict = dict(zip(train_h2o['nom_5'], train_h2o['nom_5_te']))
train_h2o_nom_6_dict = dict(zip(train_h2o['nom_6'], train_h2o['nom_6_te']))
train_h2o_nom_7_dict = dict(zip(train_h2o['nom_7'], train_h2o['nom_7_te']))
train_h2o_nom_8_dict = dict(zip(train_h2o['nom_8'], train_h2o['nom_8_te']))
train_h2o_nom_9_dict = dict(zip(train_h2o['nom_9'], train_h2o['nom_9_te']))

In [225]:
# добавляем столбцы nom_i_te (для симметрии с train_h2o) и преобразуем в них категории

test_h2o['nom_5_te'] = test_h2o['nom_5']
test_h2o['nom_5_te'] = test_h2o['nom_5_te'].map(train_h2o_nom_5_dict)
test_h2o['nom_6_te'] = test_h2o['nom_6']
test_h2o['nom_6_te'] = test_h2o['nom_6_te'].map(train_h2o_nom_6_dict)
test_h2o['nom_7_te'] = test_h2o['nom_7']
test_h2o['nom_7_te'] = test_h2o['nom_7_te'].map(train_h2o_nom_7_dict)
test_h2o['nom_8_te'] = test_h2o['nom_8']
test_h2o['nom_8_te'] = test_h2o['nom_8_te'].map(train_h2o_nom_8_dict)
test_h2o['nom_9_te'] = test_h2o['nom_9']
test_h2o['nom_9_te'] = test_h2o['nom_9_te'].map(train_h2o_nom_9_dict)

In [226]:
# очевидные преобразования порядковых признаков ord_0, ord_1, ord_2

map_ord0 = {1.0:0, 2.0:1, 3.0:2}
map_ord1 = {'Novice':0, 'Contributor':1,'Expert':2,'Master':3, 'Grandmaster':4}
map_ord2 = {'Freezing':0, 'Cold':1, 'Warm':2, 'Hot':3, 'Boiling Hot':4, 'Lava Hot':5}
train_h2o['ord_0'] = train_h2o['ord_0'].map(map_ord0)
train_h2o['ord_1'] = train_h2o['ord_1'].map(map_ord1)
train_h2o['ord_2'] = train_h2o['ord_2'].map(map_ord2)
test_h2o['ord_0'] = test_h2o['ord_0'].map(map_ord0)
test_h2o['ord_1'] = test_h2o['ord_1'].map(map_ord1)
test_h2o['ord_2'] = test_h2o['ord_2'].map(map_ord2)

In [227]:
# преобразование порядковых признаков ord_3, ord_4, ord_5 по алфавиту

le_3 = LabelEncoder().fit(train_h2o['ord_3'])
train_h2o['ord_3'] = le_3.transform(train_h2o['ord_3'])
test_h2o['ord_3'] = le_3.transform(test_h2o['ord_3'])
le_4 = LabelEncoder().fit(train_h2o['ord_4'])
train_h2o['ord_4'] = le_4.transform(train_h2o['ord_4'])
test_h2o['ord_4'] = le_4.transform(test_h2o['ord_4'])
le_5 = LabelEncoder().fit(train_h2o['ord_5'])
train_h2o['ord_5'] = le_5.transform(train_h2o['ord_5'])
test_h2o['ord_5'] = le_5.transform(test_h2o['ord_5'])

In [228]:
# преобразование циклических признаков day, month

train_h2o['sin_day'] = np.sin(2 * np.pi * train_h2o.day / 7)
train_h2o['cos_day'] = np.cos(2 * np.pi * train_h2o.day / 7)
test_h2o['sin_day'] = np.sin(2 * np.pi * test_h2o.day / 7)
test_h2o['cos_day'] = np.cos(2 * np.pi * test_h2o.day / 7)
train_h2o['sin_month'] = np.sin(2 * np.pi * train_h2o.month / 12)
train_h2o['cos_month'] = np.cos(2 * np.pi * train_h2o.month / 12)
test_h2o['sin_month'] = np.sin(2 * np.pi * test_h2o.month / 12)
test_h2o['cos_month'] = np.cos(2 * np.pi * test_h2o.month / 12)

In [231]:
# создадим датафреймы с общепринятыми именами, удалим лишние столбцы

X_train_raw = train_h2o.copy()
X_test_raw = test_h2o.copy()
X_train = X_train_raw.drop(['nom_5', 'nom_6', 'nom_7', 'nom_8', 'nom_9', 'day', 'month', 'cv_fold_te'], axis=1)
X_test = X_test_raw.drop(['nom_5', 'nom_6', 'nom_7', 'nom_8', 'nom_9', 'day', 'month'], axis=1)

# выделим зависимую переменную
y_train = X_train['target']
del X_train['target']


In [234]:
X_test.id

0         600000
1         600002
2         600003
3         600006
4         600008
           ...  
198157    999993
198158    999994
198159    999995
198160    999998
198161    999999
Name: id, Length: 198162, dtype: int64